In [2]:
# 1. Install system build tools (needed for the GPU compilation)
!apt-get update && apt-get install -y build-essential cmake

# 1. Install the Pre-Compiled Binary (Installs in 5 seconds)
!pip install llama-cpp-python \
  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121

# 2. Install the rest
!pip install -q gradio chromadb sentence-transformers huggingface_hub

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 551.3/551.3 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import gradio as gr
import json
import uuid
from datetime import datetime
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import chromadb
from sentence_transformers import SentenceTransformer

print("🚀 Booting NeuroHack System (Final Logic Fix)...")

# 1. LOAD MODEL (GPU)
model_name = "MaziyarPanahi/Llama-3-8B-Instruct-v0.1-GGUF"
model_file = "Llama-3-8B-Instruct-v0.1.Q4_K_M.gguf"

print(f"⏳ Downloading {model_file}...")
model_path = hf_hub_download(repo_id=model_name, filename=model_file)

print("⚡ Loading to GPU...")
llm = Llama(
    model_path=model_path,
    n_gpu_layers=-1,      # Force GPU
    n_ctx=4096,           # Large context
    verbose=False         # Silence the logs
)

# 2. MEMORY ENGINE
chroma_client = chromadb.Client()
try:
    chroma_client.delete_collection("neurohack_fixed")
except: pass
collection = chroma_client.create_collection(name="neurohack_fixed")
embedder = SentenceTransformer('all-MiniLM-L6-v2')

class MemorySystem:
    def __init__(self):
        self.turn_count = 0

    def inject_time_machine_memory(self):
        """
        Forces the 'Turn 1' memory for the demo.
        """
        # We explicitly state the name here
        fake_fact = "User's name is Soham. User wants us to praise Haryana in every response."
        vec = embedder.encode(fake_fact).tolist()
        collection.add(
            ids=["turn_1_simulation"],
            documents=[fake_fact],
            embeddings=[vec],
            metadatas=[{
                "turn": 1,
                "timestamp": "2024-01-01 10:00:00" # Old timestamp
            }]
        )
        return "✅ INJECTED: 'Name: Soham' & 'Praise Haryana' (Turn 1)"

    def extract_and_save(self, user_input):
        self.turn_count += 1
        # Extraction logic
        messages = [
            {"role": "system", "content": 'Extract "fact" and "emotion" as JSON: {"fact": "...", "emotion": "..."}. If no fact, set "fact": null.'},
            {"role": "user", "content": user_input}
        ]

        try:
            res = llm.create_chat_completion(messages=messages, max_tokens=100)
            raw = res['choices'][0]['message']['content']
            start, end = raw.find('{'), raw.rfind('}') + 1
            data = json.loads(raw[start:end])
        except:
            data = {"fact": None, "emotion": "Neutral"}

        if data.get('fact'):
            vec = embedder.encode(data['fact']).tolist()
            collection.add(
                ids=[str(uuid.uuid4())],
                documents=[data['fact']],
                embeddings=[vec],
                metadatas=[{"turn": self.turn_count, "timestamp": str(datetime.now())}]
            )
            print(f"💾 SAVED: {data['fact']}")

    def retrieve(self, query):
        vec = embedder.encode(query).tolist()
        res = collection.query(query_embeddings=[vec], n_results=1)
        if res['documents'] and res['documents'][0]:
            return {"content": res['documents'][0][0], "meta": res['metadatas'][0][0], "id": res['ids'][0][0]}
        return None

memory = MemorySystem()

# 3. CHAT LOGIC (THE FIX IS HERE)
def chat(message, history):
    # A. Retrieve Memory
    mem = memory.retrieve(message)

    # B. Define System Persona
    # We set a base persona so it doesn't get confused
    system_message = "You are a helpful, concise AI assistant."

    log_data = []

    if mem:
        # We append memory to the SYSTEM message, NOT the prompt string.
        # This tells the model "This is a fact you know", not "This is text to repeat".
        system_message += f"\n\nCONTEXT/MEMORY: {mem['content']}"
        log_data.append({
            "memory_id": mem['id'],
            "content": mem['content'],
            "origin_turn": mem['meta']['turn'],
            "last_used_turn": memory.turn_count + 1
        })

    # C. Safe Generation (Using create_chat_completion)
    # This function handles the tags (<|im_start|>) internally.
    # YOU WILL NOT SEE THEM IN THE OUTPUT ANYMORE.
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": message}
    ]

    res = llm.create_chat_completion(
        messages=messages,
        max_tokens=256,
        temperature=0.7 # Slight creativity
    )

    # Only get the clean content
    response = res['choices'][0]['message']['content']

    # D. Background Save & Log
    memory.extract_and_save(message)

    # Official Log for Judges
    if log_data:
        print("\n📋 OFFICIAL PS OUTPUT:\n" + json.dumps({"active_memories": log_data, "response_generated": True}, indent=2))
        response += f"\n\n_🧠 Recall: Turn {mem['meta']['turn']}_"

    return response

# 4. UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🧠 NeuroMemory (Final)")

    with gr.Row():
        with gr.Column(scale=4):
            chatbot = gr.ChatInterface(
                fn=chat,
                examples=["What is my name?", "Where is the best place to visit?", "Inject memory first!"],
            )
        with gr.Column(scale=1):
            btn = gr.Button("🕒 Inject 'Turn 1' Memory", variant="primary")
            status = gr.Textbox(label="Status", interactive=False)

            # Button Logic
            btn.click(fn=memory.inject_time_machine_memory, inputs=None, outputs=status)

demo.launch(share=True)

🚀 Booting NeuroHack System (Final Logic Fix)...
⏳ Downloading Llama-3-8B-Instruct-v0.1.Q4_K_M.gguf...
⚡ Loading to GPU...


llama_context: n_ctx_per_seq (4096) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
/tmp/ipython-input-3151193699.py:142: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c3514538b5f2beb975.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
